# Notebook: Split Dataset in folds

## Packages

In [1]:
from dataset_statistics import df_statistics
import json

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Parameters

In [2]:
DATASET_PATH = "dataset_total/filtered_dataset.json"

## Code

### Create Statistics

In [3]:
with open(DATASET_PATH, 'r', encoding='utf-8') as json_file:
    dataset = json.load(json_file)

df_statistics(dataset, False)

{'most_frequent_aspect_terms_FOOD': [('Suppe', 167),
  ('Kuchen', 167),
  ('Getränke', 167),
  ('Käsespätzle', 167),
  ('Essen', 166)],
 'most_frequent_tokens_in_text_FOOD': [('Suppe', 167),
  ('Kuchen', 167),
  ('Getränke', 167),
  ('Käsespätzle', 167),
  ('Essen', 166)],
 'unique_aspect_terms_FOOD': 5,
 'number_of_implicit_aspects_FOOD': 0,
 'number_of_explicit_aspects_FOOD': 834,
 'number_of_aspects_FOOD': 834,
 'most_frequent_aspect_terms_SERVICE': [('Kellner', 334),
  ('Personal', 333),
  ('Service', 167)],
 'most_frequent_tokens_in_text_SERVICE': [('Kellner', 334),
  ('Personal', 333),
  ('Service', 167)],
 'unique_aspect_terms_SERVICE': 3,
 'number_of_implicit_aspects_SERVICE': 167,
 'number_of_explicit_aspects_SERVICE': 834,
 'number_of_aspects_SERVICE': 1001,
 'most_frequent_aspect_terms_PRICE': [('Geld', 166)],
 'most_frequent_tokens_in_text_PRICE': [('Geld', 166)],
 'unique_aspect_terms_PRICE': 1,
 'number_of_implicit_aspects_PRICE': 0,
 'number_of_explicit_aspects_PRICE': 1